# Post-Quantum Cryptography - SPHINCS+ Hash-Based Signatures

In this notebook, we will explore **SPHINCS+**, a stateless hash-based digital signature scheme designed to be secure against quantum attacks. Unlike traditional public-key cryptography, SPHINCS+ relies on hash functions for its security.

SPHINCS+ uses three key hash-based components:
1. **WOTS+ (Winternitz One-Time Signature)**: A hash-based one-time signature.
2. **Hypertree (HT)**: A hierarchical hash tree used to manage multiple WOTS+ signatures.
3. **Few-time Signatures (FTS)**: Ensures statelessness by combining WOTS+ with a Merkle tree structure.

We will simulate the key parts of SPHINCS+ to demonstrate how hash functions are used to create quantum-resistant signatures.

### Step 1: WOTS+ (Winternitz One-Time Signature)
WOTS+ is a hash-based one-time signature. Here we simulate a basic WOTS+ using SHA3-256 as the hash function.


In [12]:
import hashlib
import os

def hash_function(input_data):
  return hashlib.sha3_256(input_data).digest()

seed = os.urandom(32)

message = b"Quantum-safe message"
hashed_message = hash_function(message)

# Simulate WOTS+ signature generation by hashing the message multiple times
wots_signature = []
for i in range(8):  # Simplified for demonstration (normally involves chaining)
  wots_signature.append(hash_function(hashed_message + bytes([i])))

print("WOTS+ Signature:")
for sig in wots_signature:
  print(sig.hex())

WOTS+ Signature:
50f610a3d43899ae89ea3b0bcc3f3d485ee296947a75744788aa25760f199b82
48ed375567ad9d9cd0a7b6459206b7ccc6127afbbedff356a07fab4a39b5952b
a84efc210f2d4c8745afa90ff02cf1cb0d85076fed3960885cb6bef3899b1eed
b0e450eddb57040a179301da1ae08444bcd4ceab766f5aed3f1434cb2c085b23
760b8bfb5663645593651e6f3867e6773f2d30bdf8f6d07e41af705835c38158
c08a43a58d04ea61aa4e670b72dd450f648048b9309183c4f77fcec27cdc4fb8
6635c8973d8c6c48bf2f53228f71c12ac41980c54cafe2a61be009c5cf5f627e
0608688a5af8e9faea52f30dff19e1a58bf5b6d226b2f95ae7583ffbfc43c9bf


### Step 2: Merkle Tree for Multiple Signatures
SPHINCS+ uses a Merkle tree to combine many WOTS+ signatures. Each leaf of the tree represents a WOTS+ signature, and the tree itself is used to authenticate a large number of signatures efficiently.

Here, we simulate the construction of a Merkle tree using hash functions.

In [13]:
leaf_nodes = [os.urandom(32) for _ in range(4)]  # 4 random leaves

parent_node_1 = hash_function(leaf_nodes[0] + leaf_nodes[1])
parent_node_2 = hash_function(leaf_nodes[2] + leaf_nodes[3])
merkle_root = hash_function(parent_node_1 + parent_node_2)

print("Merkle Tree Root:", merkle_root.hex())

Merkle Tree Root: f9b6e0ddb05dbc239ca0bb72cbcb7eeea144bda6d9cb529cd48ae11c28181871


### Step 3: SPHINCS+ Signature Generation
In SPHINCS+, the final signature is a combination of a WOTS+ signature, the authentication path in the Merkle tree, and the message hash.


In [14]:
# SPHINCS+ signature consists of WOTS+ signature and Merkle authentication path
sphincs_signature = {
  "wots_signature": wots_signature,
  "merkle_auth_path": [parent_node_1.hex(), parent_node_2.hex()],
  "message_hash": hashed_message.hex()
}

# Display SPHINCS+ signature
print("SPHINCS+ Signature:")
print(sphincs_signature)

SPHINCS+ Signature:
{'wots_signature': [b'P\xf6\x10\xa3\xd48\x99\xae\x89\xea;\x0b\xcc?=H^\xe2\x96\x94zutG\x88\xaa%v\x0f\x19\x9b\x82', b'H\xed7Ug\xad\x9d\x9c\xd0\xa7\xb6E\x92\x06\xb7\xcc\xc6\x12z\xfb\xbe\xdf\xf3V\xa0\x7f\xabJ9\xb5\x95+', b'\xa8N\xfc!\x0f-L\x87E\xaf\xa9\x0f\xf0,\xf1\xcb\r\x85\x07o\xed9`\x88\\\xb6\xbe\xf3\x89\x9b\x1e\xed', b'\xb0\xe4P\xed\xdbW\x04\n\x17\x93\x01\xda\x1a\xe0\x84D\xbc\xd4\xce\xabvoZ\xed?\x144\xcb,\x08[#', b'v\x0b\x8b\xfbVcdU\x93e\x1eo8g\xe6w?-0\xbd\xf8\xf6\xd0~A\xafpX5\xc3\x81X', b'\xc0\x8aC\xa5\x8d\x04\xeaa\xaaNg\x0br\xddE\x0fd\x80H\xb90\x91\x83\xc4\xf7\x7f\xce\xc2|\xdcO\xb8', b'f5\xc8\x97=\x8clH\xbf/S"\x8fq\xc1*\xc4\x19\x80\xc5L\xaf\xe2\xa6\x1b\xe0\t\xc5\xcf_b~', b'\x06\x08h\x8aZ\xf8\xe9\xfa\xeaR\xf3\r\xff\x19\xe1\xa5\x8b\xf5\xb6\xd2&\xb2\xf9Z\xe7X?\xfb\xfcC\xc9\xbf'], 'merkle_auth_path': ['9fa42a2807a9d04e1c346f50b700165eeff0634022307aa63f81b6acea209238', '5379ed62b7ebe80a7389a0fd37e7dc81a776a798ef2b36ffb4761ae18f458d26'], 'message_hash': 'ccc1c798918d60e

### Step 4: Verifying SPHINCS+ Signature
To verify the SPHINCS+ signature, we need to:
1. Recompute the WOTS+ signature from the message.
2. Verify the authentication path in the Merkle tree.
3. Compare the computed values with the signature.


In [15]:
# Recompute WOTS+ signature from message hash
recomputed_wots_signature = []
for i in range(8):
  recomputed_wots_signature.append(hash_function(hashed_message + bytes([i])))

# Recompute Merkle tree root from authentication path
recomputed_parent_node_1 = hash_function(leaf_nodes[0] + leaf_nodes[1])
recomputed_parent_node_2 = hash_function(leaf_nodes[2] + leaf_nodes[3])
recomputed_merkle_root = hash_function(recomputed_parent_node_1 + recomputed_parent_node_2)

# Check if the signature is valid
if sphincs_signature["merkle_auth_path"][0] == recomputed_parent_node_1.hex() and sphincs_signature["merkle_auth_path"][1] == recomputed_parent_node_2.hex() and recomputed_merkle_root == merkle_root:
  print("SPHINCS+ Signature Verified!")
else:
  print("Signature Verification Failed.")

SPHINCS+ Signature Verified!


### Conclusion
In this notebook, we demonstrated a simplified version of **SPHINCS+**, a hash-based post-quantum signature scheme. SPHINCS+ relies on the hardness of hash function operations (such as SHA3-256) to ensure security against both classical and quantum attacks.

By using WOTS+ for one-time signatures and combining them with Merkle trees for efficient authentication, SPHINCS+ achieves both security and performance without relying on traditional number-theoretic assumptions.

## Why is it resistant to Quantum Attacks?

To maintain security, SPHINCS+ uses multiple layers of hashes, including in the WOTS+ signature scheme and in the Merkle tree. This means even with Grover’s algorithm, a quantum computer would still need an infeasible amount of computational power to break the system because there are too many hashes to brute force.

SPHINCS+ does not rely on the product of two large primes like RSA nor uses eliptic curves like ECC. Instead, it is built entirely on hash functions, which are not susceptible to Shor’s algorithm. Shor’s algorithm specifically targets problems like integer factorization and discrete logarithms, so it doesn't apply to the hash-based structure of SPHINCS+.